In [73]:
import pandas as pd
import numpy as np
from datetime import datetime
from nbainjuries import injury
from datetime import datetime, timedelta
import warnings
import duckdb
import os
import requests

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

year = 2025
now = str((datetime.now() + timedelta(hours=3)).date())
print(f"Today's date:", now)

con = duckdb.connect(database=":memory:")
categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']

folders = os.listdir('../tables/')
df = pd.DataFrame()
for yr in folders:
    df_temp = pd.read_csv(f"../tables/{yr}/season_gamelogs.csv")
    df_temp.insert(0, 'Season', int(yr))
    df = pd.concat([df, df_temp])
df['Date'] = pd.to_datetime(df.Date)
# df = df[(df.Date != now)]
df = df.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
df['STL_BLK'] = df.STL + df.BLK
df['PR'] = df.PTS + df.REB 
df['PA'] = df.PTS + df.AST
df['RA'] = df.REB + df.AST
df['PRA'] = df.PTS + df.REB + df.AST
df

Today's date: 2025-11-20


,Season,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,TPM,TPA,3P%,FT,FTA,FT%,ORB,DRB,REB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts,STL_BLK,PR,PA,RA,PRA
0,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Christian Braun,SG,1,19.33,2,5,0.400,0,1,0.000,1,2,0.500,1,2,3,2,0,1,1,1,5,3.9,5,0,0,8.5,1,8,7,5,10
1,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Kentavious Caldwell-Pope,SG,1,36.25,8,12,0.667,2,3,0.667,2,2,1.000,1,1,2,1,3,1,3,5,20,15.2,10,0,0,19.0,4,22,21,3,23
2,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Collin Gillespie,PG,1,0.72,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0
3,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Aaron Gordon,PF,1,34.98,7,11,0.636,1,2,0.500,0,0,0.000,2,5,7,5,2,1,0,0,15,19.2,6,0,0,26.0,3,22,20,12,27
4,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Justin Holiday,SG,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,2025,20251118_PHO_POR,2025-11-18,POR,Home,PHO,Away,Damian Lillard,PG,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0
5630,2025,20251118_PHO_POR,2025-11-18,POR,Home,PHO,Away,Caleb Love,NaN,1,32.12,7,14,0.500,3,8,0.375,0,0,0.000,1,6,7,3,1,0,3,1,17,12.2,8,0,0,19.0,1,24,20,10,27
5631,2025,20251118_PHO_POR,2025-11-18,POR,Home,PHO,Away,Sidy Cissoko,NaN,1,28.12,2,7,0.286,1,4,0.250,3,3,1.000,1,2,3,5,2,0,2,2,8,7.9,2,0,0,14.5,2,11,13,8,16
5632,2025,20251118_PHO_POR,2025-11-18,PHO,Away,POR,Home,Isaiah Livers,NaN,1,12.63,1,3,0.333,1,3,0.333,2,3,0.667,0,2,2,0,0,0,0,3,5,2.3,3,0,0,5.0,0,7,5,2,7


In [74]:
%run ./common_utils.ipynb

In [75]:
df_gms = pd.read_csv(f"../tables/{year}/nba_schedule.csv")
df_gms = df_gms[(df_gms.Date == now)]
tms_today = df_gms.AwayABV.tolist() + df_gms.HomeABV.tolist()
display(df_gms)

,Date,StartTime_ET,AwayTeam,AwayABV,HomeTeam,HomeABV,Arena,AwayB2B,HomeB2B,rtrvd
222,2025-11-20,7:00p,Los Angeles Clippers,LAC,Orlando Magic,ORL,Kia Center,0,0,0
223,2025-11-20,8:00p,Sacramento Kings,SAC,Memphis Grizzlies,MEM,FedExForum,1,0,0
224,2025-11-20,8:00p,Philadelphia 76ers,PHI,Milwaukee Bucks,MIL,Fiserv Forum,1,0,0
225,2025-11-20,8:00p,Atlanta Hawks,ATL,San Antonio Spurs,SAS,Frost Bank Center,0,0,0


In [76]:
df_teams = pd.read_csv("../src/team_info_xref.csv")

df_inj = injury.get_reportdata(datetime.now() + timedelta(hours=3), return_df=True)
df_inj = df_inj.rename(columns={"Game Date": "Date", "Player Name": "Player", "Current Status": "Status"})
df_inj['Player'] = df_inj.Player.str.split(",").str[1] + " " + df_inj.Player.str.split(",").str[0]
df_inj['Player'] = df_inj['Player'].str.strip()
df_inj['Date'] = pd.to_datetime(df_inj['Date'])
df_inj = con.execute(f"""SELECT Date, ABV as Team, Player, Status, Reason FROM df_inj 
                         JOIN df_teams ON df_inj.Team = df_teams.Team
                         WHERE Date = '{now}' AND Status = 'Out'""").fetchdf()

df_best_out = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Fpts DESC) as Off_Rk FROM
                              (SELECT Team, Player, Pos, AVG(Fpts) as Fpts FROM df 
                              WHERE Season = 2025 AND Fpts > 0 AND Pos != 'None'
                              GROUP BY Team, Player, Pos)""").fetchdf()
df_best_out = con.execute(f"""SELECT * EXCLUDE(t2.Team, t2.Player) FROM df_best_out t1 
                              JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                              ORDER BY Fpts DESC""").fetchdf()
display(df_best_out)

Validated Injury-Report_2025-11-20_12AM.


,Team,Player,Pos,Fpts,Off_Rk,Date,Status,Reason
0,SAS,Victor Wembanyama,C,37.833333,2,2025-11-20,Out,Injury/Illness - Left Calf; Strain
1,MIL,Giannis Antetokounmpo,PF,35.461538,1,2025-11-20,Out,Injury/Illness - Left Adductor; Strain
2,ORL,Paolo Banchero,PF,25.125000,8,2025-11-20,Out,Injury/Illness - Left Groin; Strain
3,SAS,Stephon Castle,PG,23.115385,12,2025-11-20,Out,Injury/Illness - Left Hip Flexor; Strain
4,ATL,Trae Young,PG,19.600000,19,2025-11-20,Out,Injury/Illness - Right Knee; MCL Sprain
5,MEM,Ja Morant,PG,19.125000,22,2025-11-20,Out,Injury/Illness - Right Calf; Strain
6,SAS,Dylan Harper,SG,14.750000,27,2025-11-20,Out,Injury/Illness - Left Calf; Strain
7,MIL,Kevin Porter Jr.,PG,10.000000,48,2025-11-20,Out,Injury/Illness - Right Knee; Meniscus Surgery
8,MIL,Taurean Prince,SF,8.928571,53,2025-11-20,Out,Injury/Illness - Neck; Surgery
9,SAS,Jordan McLaughlin,PG,7.583333,53,2025-11-20,Out,Injury/Illness - Left Hamstring; Strain


In [85]:
def get_sportsbook():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://sportsbook.draftkings.com/",
        "Origin": "https://sportsbook.draftkings.com"
    }

    # dk_cats = {"PTS": 12488, "AST": 12495, "REB": 12492, "STL": , "BLK": , "STL_BLK": , "TPM": 12497, 
    #            "PA": 9973, "PR": 9976, "RA": 9974, "PRA": 5001}
    dk_cats = {"PTS": 12488, "AST": 12495, "REB": 12492, "TPM": 12497, 
               "PA": 9973, "PR": 9976, "RA": 9974, "PRA": 5001}
    df_lines = pd.DataFrame()
    for key, value in dk_cats.items():
        response = requests.get(f"https://sportsbook-nash.draftkings.com/sites/US-SB/api/sportscontent/controldata/league/leagueSubcategory/v1/markets?isBatchable=false&templateVars=42648%2C{value}&eventsQuery=%24filter%3DleagueId%20eq%20%2742648%27%20AND%20clientMetadata%2FSubcategories%2Fany%28s%3A%20s%2FId%20eq%20%27{value}%27%29&marketsQuery=%24filter%3DclientMetadata%2FsubCategoryId%20eq%20%27{value}%27%20AND%20tags%2Fall%28t%3A%20t%20ne%20%27SportcastBetBuilder%27%29&include=Events&entity=events", headers=headers)
        if response.status_code != 200:
            raise Exception('Bad Request')

        plyr_names = []
        pnt_lines = []
        for i in response.json()['selections']:
            plyr_names.append(i['participants'][0]['name'])
            pnt_lines.append(i['points'])
        df_dk = pd.DataFrame({"Player": plyr_names, f"{key}_line": pnt_lines}).drop_duplicates().reset_index(drop=True)

        if df_lines.empty:
            df_lines = df_dk
        else:
            df_lines = pd.merge(df_lines, df_dk, on="Player", how="outer")

    df_lines = pd.merge(pd.read_csv(f"../tables/{year}/plyr_pos_xref.csv"), df_lines, on='Player', how='right')
    df_lines.insert(0, 'Date', pd.to_datetime(now))

    # DELETE
    for stat in ['STL', 'BLK', 'STL_BLK']:
        df_lines[f'{stat}_line'] = np.nan

    partition_save_df(df_lines, f"../tables/{year}/parlay_lines.csv") 
    display(df_lines)

    return df_lines

df_lines = get_sportsbook()

../tables/2025/parlay_lines.csv saved!


,Date,Team,Player,Pos,PTS_line,AST_line,REB_line,TPM_line,PA_line,PR_line,RA_line,PRA_line,STL_line,BLK_line,STL_BLK_line
0,2025-11-20,MIL,AJ Green,SG,9.5,NaN,2.5,2.5,NaN,12.5,NaN,NaN,NaN,NaN,NaN
1,2025-11-20,PHI,Andre Drummond,C,9.5,NaN,10.5,NaN,NaN,19.5,NaN,NaN,NaN,NaN,NaN
2,2025-11-20,ORL,Anthony Black,PG,12.5,2.5,3.5,NaN,14.5,16.5,6.5,18.5,NaN,NaN,NaN
3,2025-11-20,MIL,Bobby Portis,PF,13.5,NaN,7.5,NaN,NaN,20.5,NaN,NaN,NaN,NaN,NaN
4,2025-11-20,LAC,Bogdan Bogdanovic,SG,11.5,2.5,3.5,1.5,14.5,15.5,6.5,17.5,NaN,NaN,NaN
5,2025-11-20,MEM,Cedric Coward,SG,15.5,3.5,7.5,1.5,19.5,NaN,NaN,NaN,NaN,NaN,NaN
6,2025-11-20,MIL,Cole Anthony,PG,10.5,4.5,3.5,1.5,14.5,13.5,7.5,17.5,NaN,NaN,NaN
7,2025-11-20,SAS,De'Aaron Fox,PG,25.5,7.5,3.5,2.5,33.5,29.5,11.5,37.5,NaN,NaN,NaN
8,2025-11-20,SAC,DeMar DeRozan,PF,17.5,3.5,3.5,NaN,20.5,20.5,6.5,24.5,NaN,NaN,NaN
9,2025-11-20,SAC,Dennis Schroder,PG,12.5,4.5,3.5,1.5,16.5,15.5,7.5,20.5,NaN,NaN,NaN


In [78]:
def pick_finder(stat, collect=False):
    df_mtch = df_gms[['AwayABV', 'HomeABV', 'AwayB2B', 'HomeB2B']]
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch = pd.concat([df_mtch, df_mtch2])
    
    df_rk = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Off_{stat} DESC) as Off_Rk FROM
                            (SELECT Team, Player, Pos, AVG({stat}) as Off_{stat} FROM df 
                            WHERE Season = 2025 AND {stat} > 0 AND Pos != 'None'
                            AND Team IN ({str(tms_today).replace("[", "").replace("]", "")})
                            GROUP BY Team, Player, Pos)""").fetchdf()
    df_save = pd.DataFrame()
    for pos in ['PG', 'SG', 'SF', 'PF', 'C']:
        print(pos)
        df_def = con.execute(f"""SELECT Team, '{pos}' AS Pos, Def_{stat}, Def_Rk FROM
                                 (SELECT *, RANK() OVER (ORDER BY Def_{stat}) as Def_Rk FROM
                                 (SELECT Opp as Team, AVG({stat}) as Def_{stat} FROM df 
                                 WHERE MP >= 27 AND Season = 2025 AND Pos = '{pos}'
                                 GROUP BY Opp
                                 ORDER BY AVG(Fpts) DESC))
                                 WHERE Team IN ({str(tms_today).replace("[", "").replace("]", "")})""").fetchdf()
        
        # Piece together the current matchups with offensive rankings vs defensive rankings
        df_picks = con.execute(f"""SELECT df_mtch.*, df_rk.* EXCLUDE(Team) FROM df_mtch 
                                  JOIN df_rk ON df_mtch.Team = df_rk.Team
                                  WHERE Pos = '{pos}'""").fetchdf()
        df_picks = con.execute(f"""SELECT df_picks.* EXCLUDE(Opp, Pos), Opp, df_def.Def_{stat}, df_def.Def_Rk FROM df_picks 
                                   JOIN df_def ON df_picks.Opp = df_def.Team""").fetchdf()
        df_picks['Rk_Diff'] = df_picks['Def_Rk'] - df_picks['Off_Rk']
        df_picks = con.execute(f"""SELECT t1.* EXCLUDE(Rk_Diff) FROM df_picks t1
                                LEFT JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                                WHERE Status IS NULL
                                ORDER BY Rk_Diff DESC""").fetchdf()
        
        # head to head
        df_h2h = pd.DataFrame()
        for index, row in df_picks.iterrows():
            query = f"""SELECT * EXCLUDE(Gms) FROM
                       (SELECT Player, COUNT(*) as Gms, AVG({stat}) AS AVG_{stat}_H2H FROM df 
                       WHERE Player = ? AND Opp = ? AND Date >= '2023-10-23'
                       GROUP BY Player)
                       WHERE Gms >= 4"""
            df_temp = con.execute(query, [row["Player"], row["Opp"]]).fetchdf()
            df_h2h = pd.concat([df_h2h, df_temp])
        try:
            df_picks = con.execute(f"""SELECT Team, B2B, t1.Player, Off_{stat}, Off_Rk, t2.AVG_{stat}_H2H, 
                                   t1.* EXCLUDE(Team, B2B, Player, Off_{stat}, Off_Rk)
                                   FROM df_picks t1 LEFT JOIN df_h2h t2 
                                   ON t1.Player = t2.Player""").fetchdf()
        except:
            pass
        if collect == False:
            add_where = "WHERE Off_Rk <= 35 AND Def_Rk >= 24"
        else:
            add_where = ""
        df_picks = con.execute(f"""SELECT df_picks.*, df_lines.{stat}_line FROM df_picks LEFT JOIN df_lines 
                                   ON df_picks.Team = df_lines.Team AND df_picks.Player = df_lines.Player
                                   {add_where}""").fetchdf()
        df_picks[f'{stat}_Diff'] = df_picks[f'Off_{stat}'] - df_picks[f'{stat}_line']
        df_picks[f'{stat}_Diff2'] = abs(df_picks[f'{stat}_Diff'])
        df_picks = df_picks.sort_values(f'{stat}_Diff2', ascending=False).reset_index(drop=True).drop(f'{stat}_Diff2', axis=1)
        df_save = pd.concat([df_save, df_picks], ignore_index=True)
        if collect == False:
            display(df_picks)
    if collect == True:
        df_save.insert(0, 'Date', pd.to_datetime(now))
        return df_save

In [71]:
for i in categories:
    print(f"==={i}===")
    pick_finder(i)

===PTS===
PG


,Team,B2B,Player,Off_PTS,Off_Rk,AVG_PTS_H2H,Opp,Def_PTS,Def_Rk,PTS_line,PTS_Diff
0,SAC,1,Russell Westbrook,14.384615,8,12.714286,MEM,23.818182,28,13.5,0.884615
1,SAC,1,Dennis Schroder,13.230769,9,15.800000,MEM,23.818182,28,12.5,0.730769
2,SAC,1,Devin Carter,5.666667,16,NaN,MEM,23.818182,28,NaN,NaN
3,ATL,0,Vit Krejci,12.727273,10,NaN,SAS,23.636364,27,NaN,NaN


SG


,Team,B2B,Player,Off_PTS,Off_Rk,AVG_PTS_H2H,Opp,Def_PTS,Def_Rk,PTS_line,PTS_Diff
0,MEM,1,Vince Williams Jr.,9.555556,13,0.500000,SAC,19.909091,28,13.5,-3.944444
1,MEM,1,Cedric Coward,14.333333,7,NaN,SAC,19.909091,28,15.5,-1.166667
2,MEM,1,Jaylen Wells,9.466667,14,NaN,SAC,19.909091,28,10.5,-1.033333
3,PHI,1,VJ Edgecombe,15.461538,5,NaN,MIL,18.666667,25,14.5,0.961538
4,MEM,1,Kentavious Caldwell-Pope,8.214286,18,10.250000,SAC,19.909091,28,8.5,-0.285714
5,PHI,1,Quentin Grimes,16.769231,4,9.166667,MIL,18.666667,25,16.5,0.269231
6,MEM,1,Cam Spencer,9.133333,15,NaN,SAC,19.909091,28,NaN,NaN
7,MEM,1,John Konchar,4.428571,25,2.800000,SAC,19.909091,28,NaN,NaN
8,PHI,1,Eric Gordon,8.000000,20,5.000000,MIL,18.666667,25,NaN,NaN


SF


...

,Team,B2B,Player,Off_STL_BLK,Off_Rk,AVG_STL_BLK_H2H,Opp,Def_STL_BLK,Def_Rk,STL_BLK_line,STL_BLK_Diff


PF


,Team,B2B,Player,Off_STL_BLK,Off_Rk,AVG_STL_BLK_H2H,Opp,Def_STL_BLK,Def_Rk,STL_BLK_line,STL_BLK_Diff
0,MIL,1,Bobby Portis,2.000000,5,1.500000,PHI,2.428571,28,NaN,NaN
1,SAC,1,DeMar DeRozan,1.636364,14,1.000000,MEM,2.333333,25,NaN,NaN
2,MIL,1,Kyle Kuzma,1.900000,9,1.714286,PHI,2.428571,28,NaN,NaN


C


,Team,B2B,Player,Off_STL_BLK,Off_Rk,AVG_STL_BLK_H2H,Opp,Def_STL_BLK,Def_Rk,STL_BLK_line,STL_BLK_Diff
0,SAC,1,Domantas Sabonis,1.666667,15,0.833333,MEM,2.857143,24,NaN,NaN
1,MIL,1,Myles Turner,3.214286,2,2.428571,PHI,3.250000,28,NaN,NaN
2,MEM,1,Zach Edey,3.000000,3,NaN,SAC,3.200000,27,NaN,NaN
3,MEM,1,Jock Landale,1.545455,19,0.600000,SAC,3.200000,27,NaN,NaN
4,MIL,1,Jericho Sims,1.500000,20,0.384615,PHI,3.250000,28,NaN,NaN
5,SAC,1,Drew Eubanks,2.333333,7,0.400000,MEM,2.857143,24,NaN,NaN
6,MEM,1,Jaren Jackson Jr.,2.888889,5,1.666667,SAC,3.200000,27,NaN,NaN
7,SAC,1,Precious Achiuwa,1.600000,16,2.750000,MEM,2.857143,24,NaN,NaN
8,SAC,1,Maxime Raynaud,1.333333,21,NaN,MEM,2.857143,24,NaN,NaN


In [84]:
df_stats = pd.DataFrame()
for i in categories:
    print(f"==={i}===")
    df_temp = pick_finder(i, collect=True)
    df_stats = pd.concat([df_stats, df_temp], ignore_index=True)
    
df_stats = con.execute("SELECT Date, Team, B2B, Player, Opp, * EXCLUDE(Date, Team, B2B, Player, Opp, Off_Rk, Def_Rk) FROM df_stats").fetchdf()
df_save = pd.DataFrame()
for player in df_stats.Player.unique():
    df_temp = df_stats[(df_stats.Player == player)]
    for col in df_temp.columns.difference(['Date', 'Team', 'Player']):
        df_temp[col] = df_temp[col].ffill()
        df_temp[col] = df_temp[col].backfill()
    df_temp = df_temp.drop_duplicates()
    df_save = pd.concat([df_save, df_temp])
print('Saving for Date:', now)
partition_save_df(df_save, f"../tables/{year}/parlay_stats.csv")
display(df_save)

===PTS===
PG
SG
SF
PF
C
===AST===
PG
SG
SF
PF
C
===REB===
PG
SG
SF
PF
C
===PR===
PG
SG
SF
PF
C
===PA===
PG
SG
SF
PF
C
===RA===
PG
SG
SF
PF
C
===PRA===
PG
SG
SF
PF
C
===TPM===
PG
SG
SF
PF
C
===STL===
PG
SG
SF
PF
C
===BLK===
PG
SG
SF
PF
C
===STL_BLK===
PG
SG
SF
PF
C
Saving for Date: 2025-11-20
../tables/2025/parlay_stats.csv saved!


,Date,Team,B2B,Player,Opp,Off_PTS,AVG_PTS_H2H,Def_PTS,PTS_line,PTS_Diff,Off_AST,AVG_AST_H2H,Def_AST,AST_line,AST_Diff,Off_REB,AVG_REB_H2H,Def_REB,REB_line,REB_Diff,Off_PR,AVG_PR_H2H,Def_PR,PR_line,PR_Diff,Off_PA,AVG_PA_H2H,Def_PA,PA_line,PA_Diff,Off_RA,AVG_RA_H2H,Def_RA,RA_line,RA_Diff,Off_PRA,AVG_PRA_H2H,Def_PRA,PRA_line,PRA_Diff,Off_TPM,AVG_TPM_H2H,Def_TPM,TPM_line,TPM_Diff,Off_STL,AVG_STL_H2H,Def_STL,STL_line,STL_Diff,Off_BLK,AVG_BLK_H2H,Def_BLK,BLK_line,BLK_Diff,Off_STL_BLK,AVG_STL_BLK_H2H,Def_STL_BLK,STL_BLK_line,STL_BLK_Diff
0,2025-11-20,MIL,1,Ryan Rollins,PHI,17.400000,4.166667,18.090909,22.5,-5.100000,5.333333,1.500000,5.818182,5.5,-0.166667,4.000000,1.500000,5.545455,3.5,0.500000,21.133333,5.666667,23.636364,26.5,-5.366667,22.733333,5.666667,23.909091,28.5,-5.766667,9.066667,3.000000,11.363636,9.5,-0.433333,26.466667,7.166667,29.454545,32.5,-6.033333,2.846154,0.500000,1.909091,2.5,0.346154,2.083333,0.333333,1.272727,NaN,NaN,1.000000,0.000000,0.454545,NaN,NaN,2.333333,0.333333,1.727273,NaN,NaN
1,2025-11-20,PHI,1,Tyrese Maxey,MIL,32.538462,31.000000,20.642857,28.5,4.038462,7.692308,5.666667,6.000000,7.5,0.192308,4.923077,4.500000,5.571429,3.5,1.423077,37.461538,35.500000,26.214286,32.5,4.961538,40.230769,36.666667,26.642857,35.5,4.730769,12.615385,10.166667,11.571429,10.5,2.115385,45.153846,41.166667,32.214286,39.5,5.653846,3.846154,4.000000,2.000000,3.5,0.346154,1.500000,1.333333,1.428571,NaN,NaN,1.666667,0.333333,0.285714,NaN,NaN,2.153846,1.666667,1.714286,NaN,NaN
2,2025-11-20,SAS,0,De'Aaron Fox,ATL,22.666667,25.200000,19.727273,25.5,-2.833333,6.166667,6.400000,6.818182,7.5,-1.333333,3.166667,5.000000,6.545455,3.5,-0.333333,25.833333,30.200000,26.272727,29.5,-3.666667,28.833333,31.600000,26.545455,33.5,-4.666667,9.333333,11.400000,13.363636,11.5,-2.166667,32.000000,36.600000,33.090909,37.5,-5.500000,2.500000,3.400000,2.181818,2.5,0.000000,1.750000,2.800000,1.454545,NaN,NaN,1.000000,0.000000,0.545455,NaN,NaN,2.250000,2.800000,2.000000,NaN,NaN
3,2025-11-20,MIL,1,Cole Anthony,PHI,9.461538,6.857143,18.090909,10.5,-1.038462,5.214286,1.714286,5.818182,4.5,0.714286,3.285714,1.571429,5.545455,3.5,-0.214286,12.071429,8.428571,23.636364,13.5,-1.428571,14.000000,8.571429,23.909091,14.5,-0.500000,8.500000,3.285714,11.363636,7.5,1.000000,17.285714,10.142857,29.454545,18.5,-1.214286,1.571429,0.714286,1.909091,1.5,0.071429,1.250000,0.285714,1.272727,NaN,NaN,1.000000,0.142857,0.454545,NaN,NaN,1.363636,0.428571,1.727273,NaN,NaN
4,2025-11-20,ORL,0,Jalen Suggs,LAC,12.545455,NaN,22.076923,13.5,-0.954545,4.727273,NaN,5.461538,4.5,0.227273,4.000000,NaN,4.076923,4.5,-0.500000,16.545455,NaN,26.153846,17.5,-0.954545,17.272727,NaN,27.538462,18.5,-1.227273,8.727273,NaN,9.538462,8.5,0.227273,21.272727,NaN,31.615385,22.5,-1.227273,2.000000,NaN,3.153846,1.5,0.500000,2.571429,NaN,1.384615,NaN,NaN,1.000000,NaN,0.461538,NaN,NaN,2.750000,NaN,1.846154,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2025-11-20,MEM,1,Jaren Jackson Jr.,SAC,17.866667,20.000000,18.700000,NaN,NaN,1.833333,2.833333,4.100000,NaN,NaN,5.266667,3.166667,12.400000,NaN,NaN,23.133333,23.166667,31.100000,NaN,NaN,19.333333,22.833333,22.800000,NaN,NaN,6.733333,6.000000,16.500000,NaN,NaN,24.600000,26.000000,35.200000,NaN,NaN,1.923077,0.833333,0.600000,NaN,NaN,1.400000,0.833333,1.500000,NaN,NaN,2.375000,0.833333,1.700000,NaN,NaN,2.888889,1.666667,3.200000,NaN,NaN
95,2025-11-20,SAC,1,Precious Achiuwa,MEM,5.857143,8.500000,20.428571,NaN,NaN,2.500000,1.000000,3.571429,NaN,NaN,4.142857,2.500000,11.714286,NaN,NaN,10.000000,11.000000,32.142857,NaN,NaN,6.571429,9.500000,24.000000,NaN,NaN,4.857143,3.500000,15.285714,NaN,NaN,10.714286,12.000000,35.714286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.666667,1.750000,1.428571,NaN,NaN,1.000000,1.000000,1.428571,NaN,NaN,1.600000,2.750000,2.857143,NaN,NaN
96,2025-